## Problem Statement

**In a competitive market like the restaurant industry, understanding the factors that influence business success is crucial for stakeholders. Utilizing the Yelp
dataset, this project aims to investigate the relationship between user engagement (reviews, tips, and check-ins) and business success metrics (review
count, ratings) for restaurants.**

## Research Objectives

1. **Quantify the correlation between user engagement (reviews, tips, check-ins) and review count/average star rating:** This will help us determine if
restaurants with higher user engagement experience a corresponding increase in reviews and ratings.
2. **Analyze the impact of sentiment on review count and average star rating:** We will investigate if positive sentiment in reviews and tips translates to higher star
ratings and potentially influences the total number of reviews left.
3. **Time trends in User Engagement:** We will explore if consistent user engagement over time is a stronger indicator of long-term success compared to sporadic
bursts of activity.

## Hypothesis Testing

• **Higher levels of user engagement (more reviews, tips, and check-ins) correlate with higher review counts and ratings for restaurants.**   
• **Positive sentiment expressed in reviews and tips contributes to higher overall ratings and review counts for restaurants.**   
• **Consistent engagement over time is positively associated with sustained business success for restaurants.**

## Importing Libraries

In [2]:
# Importing pandas for data manipulation and analysis
import pandas as pd

# Importing matplotlib for plotting and visualization
import matplotlib.pyplot as plt

# Importing seaborn for advanced statistical data visualization
import seaborn as sns

# Importing datetime for working with dates and times
from datetime import datetime

# Importing numpy for numerical operations and array manipulation
import numpy as np

# Importing sqlite3 for interacting with SQLite databases
import sqlite3

# Importing folium for creating interactive maps
import folium

# Importing Nominatim geocoder from geopy for converting addresses into geolocation data
from geopy.geocoders import Nominatim

# Importing LinearSegmentedColormap for creating custom color maps in matplotlib
from matplotlib.colors import LinearSegmentedColormap

# Importing display from IPython for displaying rich media in Jupyter notebooks
from IPython.display import display

# Suppressing warnings to avoid clutter in the output
import warnings
warnings.filterwarnings('ignore')

## Database Connection

In [3]:
# Establishes a connection to the Yelp database named 'yelp.db'.
# This connection object (`conn`) will be used to interact with the database.
conn = sqlite3.connect('yelp.db')

In [4]:
# Reads the names of all tables in the connected database (`conn`) into a Pandas DataFrame named `tables`.
# The `sqlite_master` table is a system table that contains metadata about the database, 
# and this query selects the `name` column from rows where the `type` is 'table'.
tables = pd.read_sql_query("SELECT name from sqlite_master where type = 'table'", conn)

In [5]:
tables

,name
0,business
1,review
2,user
3,tip
4,checkin


In [6]:
# Import libraries:
#   - json: for working with JSON data
#   - create_engine: from SQLAlchemy for interacting with relational databases
import json
from sqlalchemy import create_engine

In [7]:
# Load business data
with open('C:/Users/pande/Downloads/archive/yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
    business_data = [json.loads(line) for line in f]
business_df = pd.DataFrame(business_data)

# Load checkin data
with open('C:/Users/pande/Downloads/archive/yelp_academic_dataset_checkin.json', 'r') as f:
    checkin_data = [json.loads(line) for line in f]
checkin_df = pd.DataFrame(checkin_data)

# Load review data
with open('C:/Users/pande/Downloads/archive/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
    review_data = [json.loads(line) for line in f]
review_df = pd.DataFrame(review_data)

# Load tip data
with open('C:/Users/pande/Downloads/archive/yelp_academic_dataset_tip.json', 'r', encoding='utf-8') as f:
    tip_data = [json.loads(line) for line in f]
tip_df = pd.DataFrame(tip_data)

# Load user data
with open('C:/Users/pande/Downloads/archive/yelp_academic_dataset_user.json', 'r', encoding='utf-8') as f:
    user_data = [json.loads(line) for line in f]
user_df = pd.DataFrame(user_data)

# Print shapes of the loaded DataFrames
print("business_df shape:", business_df.shape)
print("checkin_df shape:", checkin_df.shape)
print("review_df shape:", review_df.shape)
print("tip_df shape:", tip_df.shape)
print("user_df shape:", user_df.shape)

business_df shape: (150346, 14)
checkin_df shape: (131930, 2)
review_df shape: (6990280, 9)
tip_df shape: (908915, 5)
user_df shape: (1987897, 22)


In [8]:
# Removes the 'attributes' and 'hours' columns from the business_df DataFrame in place.
# This modification is done 'in-place' which means the original business_df DataFrame is altered.
# If you prefer to keep the original DataFrame intact, you can remove the 'inplace=True' argument.
business_df.drop(['attributes', 'hours'], axis=1, inplace=True)

In [ ]:
# Create a database engine object for the 'yelp.db' SQLite database
engine = create_engine('sqlite:///yelp.db')

# Define a function to load a pandas DataFrame into a database table
def load_dataframe (df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Load each DataFrame into a separate table 
load_dataframe (business_df, 'business', engine) 
load_dataframe(review_df, 'review', engine) 
load_dataframe (user_df, 'user', engine) 
load_dataframe (tip_df, 'tip', engine)
load_dataframe (checkin_df, 'checkin', engine)

In [ ]:
# Explore what type of data available in the table
for table in tables['name']:
    display(pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5", conn))

## Data Analysis

In [ ]:
# Checking total count of all business
pd.read_sql_query("SELECT COUNT(*) FROM business", conn)

In [ ]:
# Checking total count of all restaurants which are open or closed
pd.read_sql_query("""SELECT business_id FROM business WHERE LOWER(categories) LIKE '%restaurant%' """, conn)

In [ ]:
# Checking total count of all restaurants which are OPEN 
business_id = pd.read_sql_query("""SELECT business_id, review_count FROM business WHERE LOWER(categories) LIKE '%restaurant%' AND is_open = 1""", conn)

In [ ]:
# What is the descriptive stats for review count and star rating for the business(restaurants)?
# To find descriptive stats we need AVG, MIN, MAX, MEDIAN

# Getting descriptive stats for review count and star rating
pd.read_sql_query(f"""SELECT 
AVG(review_count) AS Average_Reviews,
MIN(review_count) AS Min_Review_Count, 
MAX(review_count) AS Max_Review_Count,
(SELECT review_count FROM business ORDER BY review_count LIMIT 1 OFFSET (SELECT COUNT(*) FROM business) / 2) AS Median_Review,


AVG(stars) AS Average_Stars,
MIN(stars) AS Min_Star, 
MAX(stars) AS Max_Star,
(SELECT stars FROM business ORDER BY stars LIMIT 1 OFFSET (SELECT COUNT(*) FROM business) / 2) AS Median_Star_Rating

FROM business WHERE business_id IN {tuple(business_id['business_id'])} """, conn).transpose()

In [ ]:
# Writing a function for identifying outliers
def remove_outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    IQR = q3 - q1
    lower_bound = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

In [ ]:
business_id = remove_outliers(business_id, 'review_count')

In [ ]:
business_id.shape

In [ ]:
pd.read_sql_query(f"""SELECT 
AVG(review_count) AS Average_Reviews,
MIN(review_count) AS Min_Review, 
MAX(review_count) AS Max_Review,
(SELECT review_count FROM business ORDER BY review_count LIMIT 1 OFFSET (SELECT COUNT(*) FROM business) / 2) AS Median_Review,


AVG(stars) AS Average_Stars,
MIN(stars) AS Min_Star, 
MAX(stars) AS Max_Star,
(SELECT stars FROM business ORDER BY stars LIMIT 1 OFFSET (SELECT COUNT(*) FROM business) / 2) AS Median_Star_Rating

FROM business WHERE business_id IN {tuple(business_id['business_id'])} """, conn).transpose()

In [ ]:
# Which restaurants have the highest number of reviews?
pd.read_sql_query(f"""SELECT name, SUM(review_count) AS Total_Review, AVG(stars) AS Average_Rating
FROM business
WHERE business_id IN {tuple(business_id['business_id'])}
GROUP BY name
ORDER BY Total_Review DESC
LIMIT 10""", conn)

In [ ]:
# Which restaurants have the highest number of ratings?
pd.read_sql_query(f"""SELECT name, SUM(review_count) AS Total_Review, AVG(stars) AS Average_Rating
FROM business
WHERE business_id IN {tuple(business_id['business_id'])}
GROUP BY name
ORDER BY Average_Rating DESC
LIMIT 10""", conn)

In [ ]:
# Do restaurants with higher engagement tends to have higher ratings??

# Getting checkin count
pd.read_sql_query("""SELECT business_id,
SUM(LENGTH(date) - LENGTH(REPLACE(date,',',''))+1) AS Checkin_Count
FROM checkin
GROUP BY business_id""", conn)

In [ ]:
# Getting tip count
pd.read_sql_query("""SELECT business_id,
COUNT(*) AS Tip_Count
FROM tip
GROUP BY business_id""", conn)

In [ ]:
# This code retrieves average rating, review count, checkin count, and tip count data for a specific set of businesses.

# Inner query (aliased as 'total'):
# 1. Selects data from the 'business' table, including business ID, total review count per business, 
#    average star rating per business, and calculated check-in count based on comma separated dates in the 'checkin' table.
# 2. Left joins with the 'checkin' table to include check-in data (even for businesses without check-ins).
# 3. Left joins with a subquery that calculates tip count per business from the 'tip' table.
# 4. Filters results to only include businesses specified in the 'business_id' list.
# 5. Groups data by business ID.

# Outer query:
# 1. Selects and calculates average values from the inner query results ('total') for: average rating, 
#    review count, check-in count, and tip count.
# 2. Groups the final results by average rating.
review_count_df = pd.read_sql_query(f"""SELECT total.avg_rating as rating,
AVG(total.review_count) AS avg_review_count,
AVG(total.checkin_count) AS avg_checkin_count,
AVG(total.tip_count) AS avg_tip_count
FROM
(SELECT
    b.business_id,
    SUM(b.review_count) AS review_count,
    AVG(b.stars) AS avg_rating,
    SUM(LENGTH(cc.date) - LENGTH (REPLACE(cc.date, ',', '')) + 1) AS checkin_count,
    SUM(tip.tip_count) as tip_count
FROM    
    business b
LEFT JOIN
    checkin cc ON b.business_id = cc.business_id
LEFT JOIN
    (SELECT business_id, COUNT (business_id) as tip_count FROM tip GROUP BY business_id ORDER BY tip_count) AS tip ON b.business_id = tip.business_id
WHERE b.business_id IN {tuple(business_id ['business_id'])}
GROUP BY
    b.business_id) AS total
    
GROUP BY total.avg_rating

""",conn)

In [ ]:
# This code creates a visualization of the average review count, check-in count, 
# and tip count for businesses grouped by their average rating.

# Configure plot
plt.figure(figsize=(15, 5))  # Create a figure with desired size
plt.title('AVG Engagement based on Rating\n\n')  # Set the title

# Suppress default x and y ticks labels
plt.yticks([])
plt.xticks([])

# Subplot 1: Review Count
plt.subplot(1, 3, 1)  # Create subplot in a 1 row, 3 column grid at position 1
plt.title('Review Count')  # Set subplot title
plt.barh(  # Create horizontal bar chart
    review_count_df['rating'].astype('str'),  # X-axis: rating values as strings
    review_count_df['avg_review_count'],  # Y-axis: average review count values
    edgecolor='k', color='#CB754B'  # Bar properties: black border, orange color
)
plt.gca().spines['right'].set_visible(False)  # Hide right spine for aesthetics
# Add data labels above bars
for i, value in enumerate(review_count_df['avg_review_count']):
    plt.text(value + 3, i, str(round(value)), color='black', va='center')
plt.xticks([])  # Suppress x-axis ticks again

# Subplot 2: Checkin Count (similar structure as subplot 1)
plt.subplot(1, 3, 2)
plt.title('Checkin Count')
plt.barh(
    review_count_df['rating'].astype('str'),
    review_count_df['avg_checkin_count'],
    edgecolor='k',
    color='#F8862C'
)
plt.gca().spines['right'].set_visible(False)
for i, value in enumerate(review_count_df['avg_checkin_count']):
    plt.text(value + 3, i, str(round(value)), color='black', va='center')
plt.xticks([])

# Subplot 3: Tip Count (similar structure as subplot 1)
plt.subplot(1, 3, 3)
plt.title('Tip Count')
plt.barh(
    review_count_df['rating'].astype('str'),
    review_count_df['avg_tip_count'],
    edgecolor='k',
    color='#E54F29'
)
for i, value in enumerate(review_count_df['avg_tip_count']):
    plt.text(value + 0.05, i, str(round(value)), color='black', va='center')
plt.xticks([])

# Display the plot
plt.show()


In [ ]:
# Is there a correlation between the number of reviews, tips and check-ins for a business??
engagement_df = pd.read_sql_query(f"""SELECT
    b.business_id,
    SUM(b.review_count) AS review_count,
    AVG(b.stars) AS avg_rating,
    SUM(LENGTH(cc.date) - LENGTH (REPLACE(cc.date, ',', '')) + 1) AS checkin_count,
    SUM(tip.tip_count) as tip_count
FROM    
    business b
LEFT JOIN
    checkin cc ON b.business_id = cc.business_id
LEFT JOIN
    (SELECT business_id, COUNT (business_id) as tip_count FROM tip GROUP BY business_id ORDER BY tip_count) AS tip ON b.business_id = tip.business_id
WHERE b.business_id IN {tuple(business_id ['business_id'])}
GROUP BY
    b.business_id

""",conn).dropna()

In [ ]:
engagement_df[['review_count', 'checkin_count', 'tip_count']].corr()

In [ ]:
# Creating heatmap for showing correlation
# Creating a custom colormap
colors = ['#FFF1E5', '#F8862C', '#CB754B']
custom_cmap = LinearSegmentedColormap.from_list("mycmap", colors)

# Define the line color
linecolor = 'black'

# Create the heatmap
sns.heatmap(
    engagement_df[['review_count', 'checkin_count', 'tip_count']].corr(), 
    cmap=custom_cmap, 
    annot=True, 
    linewidths=0.5, 
    linecolor=linecolor
)


In [ ]:
# Is there a difference in the user engagement (reviews, tips and check-ins) between high rated and low rated business?
pd.read_sql_query(f"""
SELECT
    b.business_id,
    SUM(b.review_count) AS Review_Count,
    AVG(b.stars) AS AVG_Rating,
    SUM(LENGTH(cc.date) - LENGTH(REPLACE(cc.date, ',', '')) + 1) AS Checkin_Count,
    SUM(COALESCE(tip.tip_count, 0)) AS Tip_Count,
    (CASE WHEN b.stars >= 3.5 Then 'High-Rated' ELSE 'Low-Rated' End) AS Category
FROM
    business b
LEFT JOIN
    checkin cc ON b.business_id = cc.business_id
LEFT JOIN
    (SELECT business_id, COUNT(business_id) AS Tip_Count 
     FROM tip 
     GROUP BY business_id) AS Tip 
ON b.business_id = tip.business_id
WHERE b.business_id IN {tuple(business_id['business_id'])}
GROUP BY
    b.business_id
""", conn).dropna()

# Making Category column
engagement_df['Category'] = engagement_df['avg_rating'].apply(lambda x: 'High-Rated' if x >= 3.5 else 'Low-Rated')

In [ ]:
engagement_df.groupby('Category')[['review_count', 'tip_count', 'checkin_count']].mean()

In [ ]:
# Function to calculate the success score based on the avg rating and total review count 
def calculate_success_metric(df):
    success_score = []
    for index, row in df.iterrows():
        score = row['avg_rating'] * np.log(row['review_count'] + 1)
        success_score.append(score)
    return success_score

In [ ]:
# How do the success metric (review_count or avg_rating) of restaurants vary across different states and cities?
city_df = pd.read_sql_query(f"""SELECT city, state, latitude, longitude, 
AVG(stars) AS avg_rating, 
SUM(review_count) AS review_count, 
COUNT(*) AS Restaurant_Count 
FROM business
WHERE business_id IN {tuple(business_id['business_id'])}
GROUP BY state, city
ORDER BY review_count DESC
LIMIT 10""", conn)

# Creating new column "Sucess Score"
city_df['Success_Score'] = calculate_success_metric(city_df)

In [ ]:
city_df

In [ ]:
# Create a base map
m = folium.Map(location= [city_df ['latitude'].mean(), city_df ['longitude'].mean()], zoom_start=4)
                          
# Define color scale
color_scale = folium.LinearColormap(colors=['green', 'yellow', '#E54F29'],
                                    vmin=city_df ['Success_Score'].min(),
                                    vmax=city_df ['Success_Score'].max())

# Add markers to the map
for index, row in city_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=color_scale(row['Success_Score']),
        fill=True,
        fill_color=color_scale(row['Success_Score']),
        fill_opacity=0.7,
        popup=f"Success Score: {row['Success_Score']}"
        ).add_to(m)

# Add color scale to the map 
m.add_child (color_scale)

In [ ]:
#Are there any patterns in user engagement over time for successful businesses compared to less successful ones?
# Are there any seasonal trends in the user engagement for restaurants?

# This query retrieves monthly review count and tip count data for businesses with a star rating of 3.5 or higher.

# Subqueries:
#  - Review subquery: calculates monthly review counts for high-rated businesses (stars >= 3.5).
#  - Tip subquery: calculates average star rating and monthly tip counts for high-rated businesses.

# Main query: joins review and tip subqueries by month_year to get combined engagement metrics.

high_rated_engagement = pd.read_sql_query(f"""
SELECT review.month_year, review.review_count, tip.tip_count FROM
(SELECT strftime('%m-%Y', date) AS month_year, COUNT(*) AS review_count
FROM review
WHERE business_id IN {tuple(business_id['business_id'])} AND stars >= 3.5
GROUP BY month_year
ORDER BY month_year) AS review
JOIN
(SELECT AVG(b.stars), strftime('%m-%Y', tip.date) AS month_year, COUNT(*) AS tip_count
FROM tip
JOIN business AS b
ON tip.business_id = b.business_id
WHERE tip.business_id IN {tuple(business_id['business_id'])} AND b.stars >= 3.5
GROUP BY month_year
ORDER BY month_year) AS tip

ON review.month_year = tip.month_year
;""", conn)


low_rated_engagement = pd.read_sql_query(f"""
SELECT review.month_year, review.review_count, tip.tip_count FROM
(SELECT strftime('%m-%Y', date) AS month_year, COUNT(*) AS review_count
FROM review
WHERE business_id IN {tuple(business_id['business_id'])} AND stars < 3.5
GROUP BY month_year
ORDER BY month_year) AS review
JOIN
(SELECT AVG(b.stars), strftime('%m-%Y', tip.date) AS month_year, COUNT(*) AS tip_count
FROM tip
JOIN business AS b
ON tip.business_id = b.business_id
WHERE tip.business_id IN {tuple(business_id['business_id'])} AND b.stars < 3.5
GROUP BY month_year
ORDER BY month_year) AS tip
ON review.month_year = tip.month_year

;""",conn)

In [ ]:
high_rated_engagement

In [ ]:
low_rated_engagement

In [ ]:
time_rating = pd.read_sql_query(f"""SELECT strftime('%m-%Y', date) AS month_year, AVG(stars) as avg_rating
FROM review
WHERE business_id IN {tuple(business_id ['business_id'])}
GROUP BY month_year
ORDER BY month_year
;""", conn)

In [ ]:
time_rating

In [ ]:
# Convert 'month_year' column in 'time_rating' DataFrame to datetime format
time_rating['month_year'] = pd.to_datetime(time_rating['month_year'])

# Sort 'time_rating' DataFrame by 'month_year' in ascending order (oldest to newest)
time_rating.sort_values('month_year', inplace=True)

# Filter 'time_rating' DataFrame to only include rows where 'month_year' is after '2017-01-01'
time_rating = time_rating[time_rating['month_year'] > '2017-01-01']

# Similar steps applied to 'high_rated_engagement' and 'low_rated_engagement' DataFrames:

# 1. Convert 'month_year' to datetime
high_rated_engagement['month_year'] = pd.to_datetime(high_rated_engagement['month_year'])
low_rated_engagement['month_year'] = pd.to_datetime(low_rated_engagement['month_year'])

# 2. Sort by 'month_year'
high_rated_engagement.sort_values('month_year', inplace=True)
low_rated_engagement.sort_values('month_year', inplace=True)

# 3. Filter for dates after '2017-01-01'
high_rated_engagement = high_rated_engagement[high_rated_engagement['month_year'] > '2017-01-01']
low_rated_engagement = low_rated_engagement[low_rated_engagement['month_year'] > '2017-01-01']

In [ ]:
high_rated_engagement['avg_rating'] = time_rating['avg_rating'].values

In [ ]:
plt.figure(figsize = (15,8))
plt.subplot(3,1,1)
plt.title('Tip Engagement Over Time')
plt.plot(high_rated_engagement['month_year'], high_rated_engagement['tip_count'], label = 'High Rated', color = '#E54F29') 
plt.plot(low_rated_engagement ['month_year'], low_rated_engagement ['tip_count'], label ='Low Rated', color = '#F8862C') 
plt.legend()
plt.subplot(3,1,2)
plt.title('Review Engagement Over Time')
plt.plot(high_rated_engagement['month_year'], high_rated_engagement['review_count'], label = 'High Rated', color = '#E54F29') 
plt.plot(low_rated_engagement ['month_year'], low_rated_engagement ['review_count'], label = 'Low Rated', color = '#F8862C') 
plt.legend()
plt.subplot(3,1,3)
plt.title('Avg Rating Over Time')
plt.plot(time_rating['month_year'], time_rating ['avg_rating'], color = '#E54F29')
plt.tight_layout()
plt.show()

In [ ]:
tip_high_rated = high_rated_engagement [ ['month_year', 'tip_count']].set_index('month_year') 
review_high_rated = high_rated_engagement [['month_year', 'review_count']].set_index('month_year') 
rating_df = time_rating [['month_year', 'avg_rating']].set_index('month_year')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose 
multiplicative_decomposition = seasonal_decompose(tip_high_rated,
                                                  model='multiplicative', period = 12)
plt.rcParams.update({'figure.figsize': (16,12)})
multiplicative_decomposition.plot()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose 
multiplicative_decomposition = seasonal_decompose(review_high_rated,
                                                  model='multiplicative', period = 12)
plt.rcParams.update({'figure.figsize': (16,12)})
multiplicative_decomposition.plot()
plt.show()

In [ ]:
# How does the sentiment of reviews and tips (useful, funny, cool) correlate with the success metrics of restaurants?

In [ ]:
sentiment_df = pd.read_sql_query(f"""SELECT b.business_id, AVG(b.stars) AS avg_rating, SUM(b.review_count) AS review_count,
SUM(s.useful_count) AS useful_count,
SUM(s.funny_count) AS funny_count,
SUM(s.cool_count) AS cool_count
FROM
    (SELECT business_id,
    SUM(useful) AS useful_count,
    SUM(funny) AS funny_count,
    SUM(cool) AS cool_count
FROM
    review
GROUP BY business_id) AS s
JOIN business AS b on b.business_id = s.business_id

WHERE b.business_id IN {tuple(business_id ['business_id'])} 
GROUP BY b.business_id
ORDER BY review_count""", conn)

sentiment_df = remove_outliers(sentiment_df, 'review_count')
sentiment_df = remove_outliers(sentiment_df, 'useful_count')
sentiment_df = remove_outliers(sentiment_df, 'funny_count')
sentiment_df = remove_outliers(sentiment_df, 'cool_count')

In [ ]:
sentiment_df['success_score'] = calculate_success_metric(sentiment_df)

In [ ]:
sns.heatmap(sentiment_df.iloc[:, 2:].corr(), cmap = custom_cmap, annot = True, linewidths = 0.5, linecolor = 'black')
plt.show()

In [ ]:
#Is there any difference in engagement of elite users and non elite users? 
elite_df = pd.read_sql_query(f"""SELECT
    elite,
    COUNT(*) AS row_count,
    SUM(review_count) AS total_review_count
FROM
    (SELECT
        CASE
            WHEN elite = ''THEN 'Not Elite' 
            ELSE 'Elite'
        END AS elite,
        u.review_count
     FROM
        user u) AS user_elite
GROUP BY
    elite;
""", conn)

In [ ]:
query = """
SELECT 
    CASE
        WHEN elite = '' THEN 'Not Elite'
        ELSE 'Elite'
    END AS elite_status,
    COUNT(*) AS num_users
FROM 
    user
GROUP BY 
    elite_status
"""

# Execute the query and read the result into a DataFrame
user_counts_df = pd.read_sql_query(query, conn)

# Display the DataFrame to ensure it has the expected structure
print(user_counts_df)

# Assign the 'num_users' column from the user_counts_df DataFrame to the 'num_users' column in elite_df
elite_df['num_users'] = user_counts_df['num_users']

In [ ]:
# Execute your SQL query to fetch the count of users and total review count for both Elite and non-Elite categories
query = """
SELECT 
    CASE
        WHEN elite = '' THEN 'Not Elite'
        ELSE 'Elite'
    END AS elite_status,
    COUNT(*) AS num_users,
    SUM(review_count) AS total_review_count
FROM 
    user
GROUP BY 
    elite_status
"""

# Execute the query and read the result into a DataFrame
user_review_counts_df = pd.read_sql_query(query, conn)

# Display the DataFrame to ensure it has the expected structure
print(user_review_counts_df)

# Assign the 'num_users' and 'total_review_count' columns from the user_review_counts_df DataFrame to the respective columns in elite_df
elite_df['num_users'] = user_review_counts_df['num_users']
elite_df['total_review_count'] = user_review_counts_df['total_review_count']

In [ ]:
elite_df

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting User Distribution
plt.subplot(1, 2, 1)
plt.title('User Distribution')
plt.pie(elite_df['num_users'], labels=elite_df['elite'], autopct='%.2f', startangle=180, colors=['#E54F29', '#F8862C'])

# Plotting Review Distribution
plt.subplot(1, 2, 2)
plt.title('Review Distribution')
plt.pie(elite_df['total_review_count'], labels=elite_df['elite'], autopct='%.2f', startangle=90, colors=['#E54F29', '#F8862C'])

plt.show()

In [ ]:
#What are the busiest hours for restaurants?

review_engagement = pd.read_sql_query("""SELECT
    cast (strftime('%H', date) as integer)
    AS hour,
    COUNT(*) AS review_count
FROM
    review
GROUP BY
    hour;
""", conn)


tip_engagement = pd.read_sql_query("""SELECT
    cast (strftime('%H', date) as integer)
    as hour,
    COUNT(*) AS tip_count
FROM
    tip
GROUP BY
    hour;
""", conn)


# Extract hour from checkin dates
checkin = pd.read_sql_query("SELECT date FROM checkin", conn)
checkin_engagement = []
for dates in checkin['date']:
    for date_str in dates.split(','):
        date_str = date_str.strip()
        try:
            hour = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").hour
            checkin_engagement.append(hour)
        except ValueError as e:
            print(f"Error parsing date: {date_str} -> {e}")

checkin_engagement = pd.DataFrame(checkin_engagement, columns=['hour'])
checkin_engagement = checkin_engagement.groupby('hour').size().reset_index(name='checkin_count')

In [ ]:
plt.figure(figsize=(10, 6))

# Plot Tip Engagement
plt.subplot(3, 1, 1)
plt.title('Tip Engagement')
plt.bar(tip_engagement['hour'], tip_engagement['tip_count'], color='#E54F29')

# Plot Review Engagement
plt.subplot(3, 1, 2)
plt.title('Review Engagement')
plt.bar(review_engagement['hour'], review_engagement['review_count'], color='#F8862C')

# Plot Checkin Engagement
plt.subplot(3, 1, 3)
plt.title('Checkin Engagement')
plt.bar(checkin_engagement.index, checkin_engagement['checkin_count'], color='#CB754B')

plt.tight_layout()
plt.show()